## This notebook creates a CSV with the summary statistics in Table 5
The table shows N, NA, min, max, mean, median for each of fish length, weight, and height, as well as numbers of any lice on wild fish, and abundance in farms of chalimus, lep motile, lep AF, and caligus motile

In [18]:
import pandas as pd
import numpy as np
from pathlib import Path

In [19]:
# define the paths
DATA_DIR = Path('..') / 'source_data'

wild_fish_filepath = DATA_DIR / 'all_wild_fish_lice.csv'

farm_events_filepath = DATA_DIR / 'industry_farm_abundance.csv'

chart_output_dirpath = Path('..') / 'output' / 'Table_5'

In [20]:
# import the data
wild_fish_df = pd.read_csv(wild_fish_filepath)

farm_events_df = pd.read_csv(farm_events_filepath)

/tmp/ipykernel_1120342/3320784442.py:2: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  wild_fish_df = pd.read_csv(wild_fish_filepath)


## Construct a dataframe

In [21]:
# make an empty dictionary in the format pandas can make into a Dataframe
summary_dict = {'Field': [], 'Dataset': [], 'N': [], 'NA': [], 'Min': [], 'Max': [], 'Mean': [], 'Median': []}

### Wild Fish

In [22]:
# length=0, weight=0, height=0 mean null
wild_fish_df.replace({'length': 0, 'weight': 0, 'height': 0}, np.nan, inplace=True)

In [23]:
def add_wild_row(label, field, num=None):
    """Adds a row to summary_dict with summary stats for a column in wild_fish_df"""
    summary_dict['Field'].append(label)
    summary_dict['Dataset'].append(wild_fish_filepath.stem)

    if num is None:
        # N is a count of all non-null values
        summary_dict['N'].append(wild_fish_df[field].count())

        # NA is null values as % of total
        proportion = len(wild_fish_df[pd.isnull(wild_fish_df[field])]) / len(wild_fish_df)
        summary_dict['NA'].append(proportion*100)
    else:
        # special case, use the supplied num as N
        summary_dict['N'].append(num)

        # NA doesn't make sense here, so is None
        summary_dict['NA'].append(None)

    # stats
    summary_dict['Min'].append(wild_fish_df[field].min())
    summary_dict['Max'].append(wild_fish_df[field].max())
    summary_dict['Mean'].append(wild_fish_df[field].mean())
    summary_dict['Median'].append(wild_fish_df[field].median())

In [24]:
# length
add_wild_row('Length (mm)', 'length')

In [25]:
# weight
add_wild_row('Weight (g)', 'weight')

In [26]:
# height
add_wild_row('Height (mm)', 'height')

In [27]:
# all lice

# make a new column that is the number of all types of lice
wild_fish_df['num_all_lice'] = wild_fish_df.loc[:,'lep_cop':].sum(axis=1)

add_wild_row('Any louse', 'num_all_lice', num=len(wild_fish_df))

/tmp/ipykernel_1120342/1263182653.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  wild_fish_df['num_all_lice'] = wild_fish_df.loc[:,'lep_cop':].sum(axis=1)


### Farm events

In [28]:
def add_farm_row(label, field):
    """Adds a row to summary_dict for a column in farm_events_df"""
    summary_dict['Field'].append(label)
    summary_dict['Dataset'].append(farm_events_filepath.stem)

    # N is number of sampling events
    summary_dict['N'].append(len(farm_events_df))

    # NA is null values as % of total
    proportion = len(farm_events_df[pd.isnull(farm_events_df[field])]) / len(farm_events_df)
    summary_dict['NA'].append(proportion*100)

    # stats
    summary_dict['Min'].append(farm_events_df[field].min())
    summary_dict['Max'].append(farm_events_df[field].max())
    summary_dict['Mean'].append(farm_events_df[field].mean())
    summary_dict['Median'].append(farm_events_df[field].median())

In [29]:
# chalimus abundance
add_farm_row('Chalimus_ab', 'chalimus_ab')

In [30]:
# lep motile abundance
add_farm_row('Lep_motile_ab', 'lep_motile_ab')

In [31]:
# lep AF abundance
add_farm_row('Lep_AF_ab', 'lep_af_ab')

In [32]:
# Caligus motile abundance
add_farm_row('Cal_motile_ab', 'cal_motile_ab')

In [33]:
summary_df = pd.DataFrame(summary_dict)
summary_df.head(10)

,Field,Dataset,N,NA,Min,Max,Mean,Median
0,Length (mm),all_wild_fish_lice,368150,2.286312,3.0,553.0000,53.296541,47.0000
1,Weight (g),all_wild_fish_lice,229000,39.219246,0.1,1100.0000,3.018305,0.9600
2,Height (mm),all_wild_fish_lice,127844,66.067883,1.0,39.0000,8.491698,8.0000
3,Any louse,all_wild_fish_lice,376764,NaN,0.0,263.0000,0.836417,0.0000
4,Chalimus_ab,industry_farm_abundance,10159,0.000000,0.0,46.3500,0.961493,0.1667
5,Lep_motile_ab,industry_farm_abundance,10159,0.000000,0.0,53.3667,1.627923,0.5667
6,Lep_AF_ab,industry_farm_abundance,10159,0.000000,0.0,27.8310,0.766945,0.2063
7,Cal_motile_ab,industry_farm_abundance,10159,0.000000,0.0,41.5125,0.410069,0.0556


In [34]:
summary_df.to_csv(chart_output_dirpath / 'Table_5_summary_statistics.csv', index=False)